In [ ]:
import warnings
import os
import copy
from sklearnex import patch_sklearn
import numpy as np
import pandas as pd
from category_encoders import (
    BackwardDifferenceEncoder,
    BaseNEncoder,
    BinaryEncoder,
    CatBoostEncoder,
    CountEncoder,
    GLMMEncoder,
    HelmertEncoder,
    JamesSteinEncoder,
    LeaveOneOutEncoder,
    MEstimateEncoder,
    SummaryEncoder,
    TargetEncoder,
    WOEEncoder,
)
from sklearn.experimental import enable_iterative_imputer
from sklearn import set_config
from sklearn.base import clone as model_clone
from sklearn.cluster import *
from sklearn.impute import *
from sklearn.compose import *
from sklearn.cross_decomposition import *
from sklearn.decomposition import *
from sklearn.ensemble import *
from sklearn.feature_selection import *
from sklearn.gaussian_process import *
from sklearn.linear_model import *
from sklearn.metrics import *
from sklearn.model_selection import *
from sklearn.multioutput import *
from sklearn.multiclass import *
from sklearn.naive_bayes import *
from sklearn.neighbors import *
from sklearn.neural_network import *
from sklearn.pipeline import *
from sklearn.preprocessing import *
from sklearn.kernel_approximation import *
from sklearn.svm import *
from sklearn.tree import *
from sklearn.utils import *
from sklearn.dummy import *
from sklearn.semi_supervised import *
from sklearn.discriminant_analysis import *
from sklearn.covariance import *
from collections import Counter
import sklearn
from xgboost import XGBClassifier, XGBRFClassifier
from sklearn.metrics import pairwise_distances
from sklearn.metrics.pairwise import pairwise_kernels
from sklearn.calibration import *
import joblib
from pprint import pprint as pp

pd.options.compute.use_numba = True
pd.options.compute.use_numexpr = True
pd.options.compute.use_bottleneck = True
pd.options.display.max_columns = 90
set_config(display="diagram")
warnings.filterwarnings("ignore")
import pickle
from collections import defaultdict
import matplotlib.pyplot as plt
from imblearn.combine import SMOTEENN, SMOTETomek
from imblearn.under_sampling import NearMiss, RandomUnderSampler
from imblearn.pipeline import make_pipeline as make_imb_pipeline
from imblearn.ensemble import BalancedRandomForestClassifier
from imblearn.under_sampling import InstanceHardnessThreshold
from imblearn.over_sampling import (
    ADASYN,
    SMOTE,
    RandomOverSampler,
    SVMSMOTE,
    SMOTENC,
    SMOTEN,
    BorderlineSMOTE,
    KMeansSMOTE,
)

%matplotlib inline
plt.style.use("fivethirtyeight")

import seaborn as sns

sns.set()
from joblib import parallel_backend
from joblib.memory import Memory

# patch_sklearn()
KAGGLE_ENV = 1
DATA_INPUT = "/kaggle/input/marketing-strategy-personalised-offer/"
DATA_OUTPUT = "/kaggle/working/"
cwd = os.path.abspath(os.getcwd())
if "mlop3n/Pycharm" in cwd or "u164131" in cwd:
    KAGGLE_ENV = 0
    DATA_INPUT = "kaggle/input/marketing-strategy-personalised-offer/"
    DATA_OUTPUT = "kaggle/working/"
CACHE = Memory(DATA_OUTPUT + "joblib", verbose=0)
patch_sklearn()

data = pd.read_csv(DATA_INPUT + "train_data.csv")
eval_data = pd.read_csv(DATA_INPUT + "test_data.csv")

def gen_train_test(X, y, test_size):
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=test_size, stratify=y, random_state=10
    )
    return X_train, X_test, y_train, y_test


def quick_test(X):
    clfs = [
        RandomForestClassifier(class_weight="balanced_subsample", random_state=42),
        DecisionTreeClassifier(class_weight="balanced", random_state=42),
        HistGradientBoostingClassifier(random_state=42),
        LogisticRegressionCV(max_iter=10000, class_weight="balanced", random_state=42),
    ]
    y = data.target
    X_train, X_test, y_train, y_test = gen_train_test(X, y, test_size=0.5)
    for clf in clfs:
        y_pred = clf.fit(X_train, y_train).predict(X_test)
        score = f1_score(y_test, y_pred, average="macro")
        print(f"{clf.__class__.__name__} :: {score}")

In [ ]:
_ = data.isna().sum().plot(kind="bar", rot=90, logy=True, figsize=(10,8))
data.nunique()
n_c= ["Job/Job Industry","Marital Status","restaurant type","Customer type","Climate","Qualification","drop location","car"]
# for c in n_c:
#     print(c,"\t:\t",data[c].unique(),'\n')

```text
Job/Job Industry 	:	 ['Unemployed' 'Arts Design Entertainment Sports & Media' 'Sales & Related'
 'Student' 'Business & Financial' 'Computer & Mathematical'
 'Office & Administrative Support' 'Management' 'Healthcare Support'
 'Life Physical Social Science' 'Installation Maintenance & Repair'
 'Legal' 'Community & Social Services' 'Education&Training&Library'
 'Construction & Extraction' 'Healthcare Practitioners & Technical'
 'Transportation & Material Moving' 'Retired' 'Architecture & Engineering'
 'Production Occupations' 'Farming Fishing & Forestry'
 'Protective Service' 'Personal Care & Service'
 'Food Preparation & Serving Related'
 'Building & Grounds Cleaning & Maintenance'] 

Marital Status 	:	 ['Married partner' 'Single' 'Divorced' 'Unmarried partner' 'Widowed'] 

restaurant type 	:	 ['4 star restaurant' 'Take-away restaurant' 'Cold drinks'
 'Restaurant with pub' '2 star restaurant'] 

Customer type 	:	 ['Individual' 'With Family' 'With Kids' 'With Colleagues'] 

Climate 	:	 ['Spring' 'Summer' 'Winter'] 

Qualification 	:	 ['Bachelors degree' 'Some college - no degree'
 'Graduate degree (Masters or Doctorate)' 'Associates degree'
 'High School Graduate' 'Some High School'] 

drop location 	:	 ['Location B' 'Location A' 'Location C'] 

car 	:	 [nan 'Mazda5' 'Car that is too old to install Onstar :D' 'crossover'
 'Scooter and motorcycle' 'do not drive'] 

```

### Data Cleaning If possible Reload Last Saved Data

In [ ]:
data = pd.read_csv(DATA_INPUT + "train_data.csv")
eval_data = pd.read_csv(DATA_INPUT + "test_data.csv")
ques = "Do you want to use new data?"
ans = input(prompt=ques)
if ans.lower() == "y":

    def convert_to_hours(row):
        if "hours" in row["offer expiration"]:
            #         row['offer expiration'] = int(row['offer expiration'][:-5])
            row["offer expiration"] = 0
        elif "days" in row["offer expiration"]:
            #         row['offer expiration'] = int(row['offer expiration'][:-4])*24
            row["offer expiration"] = 1
        return row

    def save_data():
        global data, eval_data
        data.to_parquet(DATA_OUTPUT + "data.parquet")
        eval_data.to_parquet(DATA_OUTPUT + "eval_data.parquet")

    data = data.apply(convert_to_hours, axis=1)
    eval_data = eval_data.apply(convert_to_hours, axis=1)

    def pythonise_col_names(df: pd.DataFrame):
        col_names = list(df.columns)
        rename_stubs = {}
        for x in col_names:
            if x == "Offer Accepted":
                r = "target"
                rename_stubs[x] = r
            elif "/" in x:
                r = x.split("/")[1]
                r = r.replace(" ", "_").lower()
                rename_stubs[x] = r
            elif "-" in x:
                r = x.replace("-", "_").lower()
                r = r.replace(" ", "_").lower()
                rename_stubs[x] = r
            elif " " in x:
                r = x.replace(" ", "_").lower()
                rename_stubs[x] = r
            else:
                r = x.lower()
                rename_stubs[x] = r
        return rename_stubs

    data.rename(pythonise_col_names(data), inplace=True, axis=1)
    eval_data.rename(pythonise_col_names(eval_data), inplace=True, axis=1)
    gender_code = {"Male": 0, "Female": 1}

    def binarize_gender(row):
        global gender_code
        row["gender"] = gender_code[row["gender"]]
        return row

    data = data.apply(binarize_gender, axis=1)
    eval_data = eval_data.apply(binarize_gender, axis=1)

    ages_ord = ["below21", "21", "26", "31", "36", "41", "46", "50plus"]
    ordinal_age_codes = {x: idx for idx, x in enumerate(ages_ord)}
    income_codes = {
        "Less than ₹12500": 1,
        "₹12500 - ₹24999": 2,
        "₹25000 - ₹37499": 3,
        "₹37500 - ₹49999": 4,
        "₹50000 - ₹62499": 5,
        "₹62500 - ₹74999": 6,
        "₹75000 - ₹87499": 7,
        "₹87500 - ₹99999": 8,
        "₹100000 or More": 9,
    }
    ordinal_quali_codes = {'Bachelors degree':4,
     'Some college - no degree':2,
     'Graduate degree (Masters or Doctorate)':5,
     'Associates degree':3,
     'High School Graduate':1,
     'Some High School':0
    }
    data["ordinal__income_range"] = 0
    eval_data["ordinal__income_range"] = 0
    data["ordinal__age"] = 0
    eval_data["ordinal__age"] = 0
    data['ordinal__qualif'] =0
    eval_data['ordinal__qualif'] =0
    def codify_special_ordinal_range(row):
        global income_codes, ordinal_age_codes
        row["ordinal__income_range"] = income_codes[row["income_range"]]
        row["ordinal__age"] = ordinal_age_codes[row["age"]]
        row["ordinal__qualif"] = ordinal_quali_codes[row["qualification"]]
        return row

    data = data.apply(codify_special_ordinal_range, axis=1)
    eval_data = eval_data.apply(codify_special_ordinal_range, axis=1)

    ordinal_category_codes = {
        "4~8": 3,
        "less1": 1,
        "never": 0,
        "1~3": 2,
        "gt8": 4,
        np.nan: np.nan,
    }

    # data = data.apply(codify_cold_drink_consumption,axis=1)
    # eval_data = eval_data.apply(codify_cold_drink_consumption,axis=1)
    ord_cols = [
        "restaur_spend_less_than20",
        "no_take_aways",
        "restaur_spend_greater_than20",
        "no_visited_bars",
        "no_visited_cold_drinks",
    ]
    col_ord_names = ["ordinal__" + x for x in ord_cols]
    data[col_ord_names] = 0
    eval_data[col_ord_names] = 0

    def codify_ordinal_columns(row):
        global col_ord_names, ordinal_category_codes, ord_cols
        for ord_col, col_ord_name in zip(ord_cols, col_ord_names):
            try:
                row[col_ord_name] = int(ordinal_category_codes[row[ord_col]])
            except ValueError:
                row[col_ord_name] = ordinal_category_codes[row[ord_col]]
        return row

    data = data.apply(codify_ordinal_columns, axis=1)
    eval_data = eval_data.apply(codify_ordinal_columns, axis=1)

    """
    Car Feature:
    Label NaN as unknown for now
    """
    data.car.fillna("unknown", inplace=True)
    eval_data.car.fillna("unknown", inplace=True)

    nominal_cs = [
        "restaurant_type",
        "marital_status",
        "climate",
        "drop_location",
        "job_industry",
        "customer_type",
        "car",
    ]
    nominal_col_names = ["nominal__" + x for x in nominal_cs]
    master_nominals = pd.concat(
        [data[nominal_cs], eval_data[nominal_cs]], ignore_index=True, axis=0
    )
    nominal_encoder = {nc: {} for nc in nominal_cs}
    for c in nominal_cs:
        unique_vals = list(master_nominals[c].unique())
        nominal_codes = {x: idx for idx, x in enumerate(unique_vals)}
        nominal_encoder[c] = nominal_codes
    data[nominal_col_names] = 0
    eval_data[nominal_col_names] = 0

    def codify_nominal_columns(row):
        global nominal_col_names, nominal_encoder, nominal_cs
        for nom_col, col_nom_name in zip(nominal_cs, nominal_col_names):
            try:
                row[col_nom_name] = int(nominal_encoder[nom_col][row[nom_col]])
            except ValueError:
                row[col_nom_name] = nominal_encoder[nom_col][row[nom_col]]
        return row

    data = data.apply(codify_nominal_columns, axis=1)
    eval_data = eval_data.apply(codify_nominal_columns, axis=1)

    def drop_const_and_enc_columns(df):
        cols_to_drop = [
            "restaur_spend_less_than20",
            "no_take_aways",
            "restaur_spend_greater_than20",
            "no_visited_bars",
            "no_visited_cold_drinks",
            "restaurant_type",
            "marital_status",
            "climate",
            "qualification",
            "drop_location",
            "job_industry",
            "customer_type",
            "car",
            "income_range",
            "travelled_more_than_5mins_for_offer",
            "age",
        ]
        df = df.drop(cols_to_drop, axis=1)
        return df

    data = drop_const_and_enc_columns(data)
    eval_data = drop_const_and_enc_columns(eval_data)

    data_original_col_order = [
        "offer_expiration",
        "ordinal__income_range",
        "ordinal__no_visited_cold_drinks",
        "travelled_more_than_15mins_for_offer",
        "ordinal__restaur_spend_less_than20",
        "nominal__marital_status",
        "nominal__restaurant_type",
        "ordinal__age",
        "prefer_western_over_chinese",
        "travelled_more_than_25mins_for_offer",
        "ordinal__no_visited_bars",
        "gender",
        "nominal__car",
        "restuarant_same_direction_house",
        "cooks_regularly",
        "nominal__customer_type",
        "ordinal__qualif",
        "is_foodie",
        "ordinal__no_take_aways",
        "nominal__job_industry",
        "restuarant_opposite_direction_house",
        "has_children",
        "visit_restaurant_with_rating_(avg)",
        "temperature",
        "ordinal__restaur_spend_greater_than20",
        "travel_time",
        "nominal__climate",
        "nominal__drop_location",
        "prefer_home_food",
        "target",
    ]
    eval_data_original_col_order = copy.deepcopy(data_original_col_order)
    eval_data_original_col_order.remove("target")

    data = data.loc[:, data_original_col_order]
    eval_data = eval_data.loc[:, eval_data_original_col_order]
    binary_features = [
        "offer_expiration",
        "travelled_more_than_15mins_for_offer",
        "prefer_western_over_chinese",
        "travelled_more_than_25mins_for_offer",
        "restuarant_same_direction_house",
        "cooks_regularly",
        "is_foodie",
        "restuarant_opposite_direction_house",
        "has_children",
        "prefer_home_food",
        "gender",
    ]
    b_f_rn = {x: "binary__" + x for x in binary_features}
    data.rename(b_f_rn, axis=1, inplace=True)
    eval_data.rename(b_f_rn, axis=1, inplace=True)
    remaining_ordinals = {
        "visit_restaurant_with_rating_(avg)": "ordinal__rest_rating",
        "temperature": "ordinal__temperature",
        "travel_time": "ordinal__travel_time",
    }

    def rename_remaining_ord(df):
        df = df.rename(remaining_ordinals, axis=1)
        return df

    data = rename_remaining_ord(data)
    eval_data = rename_remaining_ord(eval_data)

    target_codes = {"Yes": 1, "No": 0}

    def binarize_target(row):
        global target_codes
        row["target"] = target_codes[row["target"]]
        return row

    data = data.apply(binarize_target, axis=1)
    imputer = IterativeImputer(
        estimator=RandomForestClassifier(
            class_weight="balanced", random_state=42, n_jobs=-1
        ),
        sample_posterior=False,
        initial_strategy="most_frequent",
        random_state=42,
    )
    data.isna().sum()
    X = data[eval_data.columns]
    y = data.target
    with parallel_backend("threading", n_jobs=24):
        data_enc = imputer.fit_transform(X)
        eval_data_enc = imputer.transform(eval_data)

    data.loc[:, eval_data.columns] = data_enc
    eval_data.loc[:, :] = eval_data_enc
    data = data.astype(np.uint32)
    eval_data = eval_data.astype(np.uint32)
    save_data()
else:
    data = pd.read_parquet(DATA_OUTPUT + "data.parquet")
    eval_data = pd.read_parquet(DATA_OUTPUT + "eval_data.parquet")

In [ ]:
X = data[eval_data.columns]
y = data.target

In [ ]:
related_features = (((data.corr().abs() > 0.3).sum(axis=0)) > 1).to_dict()
relf = []
print("Features that have Linear or Non Linear Relationships\n\n".upper())
for k, v in related_features.items():
    if v:
        print("\t" + k)
        relf.append(k)
crr = data.corr()
relations = {f: {} for f in relf}
for f in relf:
    f_crr = crr[f].abs()
    #     print('\n')
    #     print(f_crr[f_crr > 0.3])
    #     print('\n')
    f_rel = crr[f][f_crr[f_crr > 0.3].index].to_dict()
    del f_rel[f]
    relations[f] = f_rel

In [ ]:
nominal = [x for x in eval_data.columns if "nominal_" in x]
ordinal = [x for x in eval_data.columns if "ordinal_" in x]
binary = [x for x in eval_data.columns if "binary_" in x]
len(nominal), len(ordinal), len(binary)
X[nominal].nunique()

In [ ]:
data.nominal__job_industry.value_counts()

In [ ]:
poly_mutex = PolynomialFeatures(degree=4, include_bias=False, interaction_only=True)
poly_mutex.fit(X)
poly_f_data = pd.DataFrame(
    poly_mutex.transform(X), columns=poly_mutex.get_feature_names_out()
)
poly_f_eval_data = pd.DataFrame(
    poly_mutex.transform(eval_data), columns=poly_mutex.get_feature_names_out()
)
vr = VarianceThreshold()
vr.fit(poly_f_data)
poly_f_data = pd.DataFrame(
    vr.transform(poly_f_data), columns=vr.get_feature_names_out()
)
poly_f_eval_data = pd.DataFrame(
    vr.transform(poly_f_eval_data), columns=vr.get_feature_names_out()
)

In [ ]:
poly_f_data.corrwith(y,method='kendall').abs().sort_values(ascending=False)

In [ ]:
with parallel_backend("threading", n_jobs=48):
    mic = chi2(poly_f_data, y)

In [ ]:
joblib.dump(mic,'mic_poly')

In [ ]:
_ = plt.plot(mic[0])
_ = plt.plot(mic[1])

In [ ]:
good_poly = np.argsort(mic[0][np.where(mic[0] > (10**5)/4)])
good_f = list(poly_f_data.columns[good_poly])

print(len(good_f))
# for e in good_f:
#     print(e)

In [ ]:
X = poly_f_data[good_f]
y = data.target
min_c = poly_f_data[good_f].nunique().astype("int").to_numpy() + 1
# clf = RandomForestClassifier(class_weight='balanced',
#                              n_jobs=24,
# #                              max_features=None,
#                              )
class_prior = (y.value_counts() / X.shape[0]).sort_index().to_numpy()
clf = CategoricalNB(
    fit_prior=True,
    alpha=0.0000003,
    min_categories=min_c,
    #                     class_prior=class_prior
)
categories_ = []
for c in X.columns:
    categories_.append(sorted(list(X[c].unique())))

work = make_pipeline(OrdinalEncoder(categories=categories_), clf)
X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.5,
        random_state=42,
    shuffle=True,
    stratify=y,
)
with parallel_backend("threading"):
    #     scores = cross_validate(clf,X,y,cv=RepeatedStratifiedKFold(n_splits=5, n_repeats=2, random_state=42),n_jobs=24,return_train_score=True,scoring='f1_macro')
    y_pred = work.fit(X_train, y_train).predict(X_test)
    print(classification_report(y_pred, y_test))

In [ ]:
X = poly_f_data[good_f]
y = data.target
clf = RandomForestClassifier(
    class_weight="balanced", n_jobs=24, max_features=None, max_depth=8
)
with parallel_backend("threading"):
    scores = cross_validate(
        clf,
        X,
        y,
        cv=RepeatedStratifiedKFold(n_splits=5, n_repeats=2, random_state=42),
        n_jobs=24,
        return_train_score=True,
        scoring="f1_macro",
    )
_ = plt.plot(scores["test_score"], label="TEST")
_ = plt.plot(scores["train_score"], label="TRAIN")
_ = plt.legend()

In [ ]:
poly_f_data.to_parquet("poly_train_4.parquet", compression="brotli")
poly_f_eval_data.to_parquet('poly_eval_4.parquet',compression="brotli")

In [ ]:
poly_f_data = pd.read_parquet("poly_train_4.parquet")

In [ ]:
poly_f_data.shape

In [ ]:
FeatureAgglomeration(n_clusters = 20,affinity=)

In [ ]:
import phik

In [ ]:
data.columns

In [ ]:
pd.options.compute.use_numba=False

In [ ]:
data_bins = data.nunique().to_dict()
phi = data.global_phik(interval_cols=ordinal,bins=data_bins,)

In [ ]:
phi

In [81]:
data.groupby('nominal__customer_type')['binary__has_children'].value_counts()

nominal__customer_type  binary__has_children
0                       0                       3582
                        1                       2349
1                       1                       1001
                        0                        646
2                       0                       1090
                        1                        529
3                       0                       1916
                        1                       1266
Name: binary__has_children, dtype: int64

In [76]:
list(eval_data.columns)

['binary__offer_expiration',
 'ordinal__income_range',
 'ordinal__no_visited_cold_drinks',
 'binary__travelled_more_than_15mins_for_offer',
 'ordinal__restaur_spend_less_than20',
 'nominal__marital_status',
 'nominal__restaurant_type',
 'ordinal__age',
 'binary__prefer_western_over_chinese',
 'binary__travelled_more_than_25mins_for_offer',
 'ordinal__no_visited_bars',
 'binary__gender',
 'nominal__car',
 'binary__restuarant_same_direction_house',
 'binary__cooks_regularly',
 'nominal__customer_type',
 'ordinal__qualif',
 'binary__is_foodie',
 'ordinal__no_take_aways',
 'nominal__job_industry',
 'binary__restuarant_opposite_direction_house',
 'binary__has_children',
 'ordinal__rest_rating',
 'ordinal__temperature',
 'ordinal__restaur_spend_greater_than20',
 'ordinal__travel_time',
 'nominal__climate',
 'nominal__drop_location',
 'binary__prefer_home_food']

In [ ]:
who = ['binary__gender','ordinal__age',]
personal_life = ['nominal__marital_status','binary__has_children','nominal__customer_type']
prefe = ['binary__prefer_home_food','binary__is_foodie','binary__prefer_western_over_chinese','binary__cooks_regularly']
distance = ['binary__travelled_more_than_15mins_for_offer','binary__travelled_more_than_25mins_for_offer','ordinal__travel_time',]
usage_info = ['ordinal__no_visited_cold_drinks','ordinal__restaur_spend_less_than20','ordinal__no_take_aways',]
